# 核主成分分析（Kernel PCA）

## 概述

核主成分分析（Kernel PCA）是 PCA 的非线性扩展，通过核技巧将数据隐式映射到高维特征空间，然后在该空间中执行 PCA。

## 核心思想

标准 PCA 只能捕捉数据中的线性结构。对于非线性可分的数据，Kernel PCA 通过以下步骤工作：

1. 使用非线性映射 $\phi(x)$ 将数据映射到高维特征空间
2. 在特征空间中执行标准 PCA
3. 利用核函数 $K(x_i, x_j) = \langle\phi(x_i), \phi(x_j)\rangle$ 避免显式计算 $\phi(x)$

## 常用核函数

- **RBF（径向基函数）核**：$K(x, y) = \exp(-\gamma\|x-y\|^2)$
- **多项式核**：$K(x, y) = (\gamma x^\top y + c_0)^d$
- **Sigmoid 核**：$K(x, y) = \tanh(\gamma x^\top y + c_0)$

## 本节内容

1. Kernel PCA 基本使用
2. 不同核函数的比较
3. 超参数调优
4. 逆变换（近似重构）

## 1. 环境准备

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import KernelPCA, PCA
from sklearn.datasets import make_moons, make_circles, make_swiss_roll
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import mean_squared_error, accuracy_score

# 设置随机种子
np.random.seed(42)

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

## 2. 生成非线性数据集

使用经典的非线性数据集来展示 Kernel PCA 的优势。

In [ ]:
# 生成三种非线性数据集
n_samples = 500

# 半月形数据
X_moons, y_moons = make_moons(n_samples=n_samples, noise=0.1, random_state=42)

# 同心圆数据
X_circles, y_circles = make_circles(n_samples=n_samples, noise=0.1, factor=0.3, random_state=42)

# 瑞士卷数据（取前两维用于演示）
X_swiss, y_swiss = make_swiss_roll(n_samples=n_samples, noise=0.5, random_state=42)
X_swiss_2d = X_swiss[:, [0, 2]]  # 选择 x 和 z 维度
y_swiss_discrete = (y_swiss > y_swiss.mean()).astype(int)

# 可视化原始数据
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

datasets = [
    (X_moons, y_moons, 'Half Moons'),
    (X_circles, y_circles, 'Concentric Circles'),
    (X_swiss_2d, y_swiss_discrete, 'Swiss Roll (2D projection)')
]

for ax, (X, y, title) in zip(axes, datasets):
    scatter = ax.scatter(X[:, 0], X[:, 1], c=y, cmap='coolwarm', 
                        alpha=0.7, edgecolors='white', linewidth=0.5)
    ax.set_title(title)
    ax.set_xlabel('Feature 1')
    ax.set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## 3. PCA vs Kernel PCA 对比

使用半月形数据集对比标准 PCA 和 Kernel PCA 的降维效果。

In [ ]:
# 标准化数据
scaler = StandardScaler()
X_moons_scaled = scaler.fit_transform(X_moons)

# 标准 PCA
pca = PCA(n_components=2)
X_pca = pca.fit_transform(X_moons_scaled)

# Kernel PCA with RBF kernel
kpca_rbf = KernelPCA(n_components=2, kernel='rbf', gamma=15)
X_kpca_rbf = kpca_rbf.fit_transform(X_moons_scaled)

# 可视化对比
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 原始数据
axes[0].scatter(X_moons_scaled[:, 0], X_moons_scaled[:, 1], 
                c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
axes[0].set_title('Original Data')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# 标准 PCA
axes[1].scatter(X_pca[:, 0], X_pca[:, 1], 
                c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
axes[1].set_title('Standard PCA')
axes[1].set_xlabel('PC1')
axes[1].set_ylabel('PC2')

# Kernel PCA
axes[2].scatter(X_kpca_rbf[:, 0], X_kpca_rbf[:, 1], 
                c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
axes[2].set_title('Kernel PCA (RBF)')
axes[2].set_xlabel('KPC1')
axes[2].set_ylabel('KPC2')

plt.tight_layout()
plt.show()

## 4. 不同核函数的比较

比较 RBF、多项式和 Sigmoid 核在同心圆数据上的表现。

In [ ]:
# 标准化同心圆数据
X_circles_scaled = StandardScaler().fit_transform(X_circles)

# 不同核函数的配置
kernels = [
    ('linear', {'kernel': 'linear'}),
    ('rbf', {'kernel': 'rbf', 'gamma': 10}),
    ('poly', {'kernel': 'poly', 'gamma': 10, 'degree': 3}),
    ('sigmoid', {'kernel': 'sigmoid', 'gamma': 1, 'coef0': 1})
]

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

for ax, (name, params) in zip(axes.flat, kernels):
    kpca = KernelPCA(n_components=2, **params)
    X_kpca = kpca.fit_transform(X_circles_scaled)
    
    ax.scatter(X_kpca[:, 0], X_kpca[:, 1], 
               c=y_circles, cmap='coolwarm', alpha=0.7, edgecolors='white')
    ax.set_title(f'Kernel: {name}')
    ax.set_xlabel('KPC1')
    ax.set_ylabel('KPC2')

plt.suptitle('Kernel PCA with Different Kernels (Concentric Circles)', fontsize=14)
plt.tight_layout()
plt.show()

## 5. Gamma 参数的影响

RBF 核的 gamma 参数控制核函数的宽度，对结果有重要影响。

In [ ]:
# 测试不同的 gamma 值
gamma_values = [0.1, 1, 5, 15, 50, 100]

fig, axes = plt.subplots(2, 3, figsize=(15, 9))

for ax, gamma in zip(axes.flat, gamma_values):
    kpca = KernelPCA(n_components=2, kernel='rbf', gamma=gamma)
    X_kpca = kpca.fit_transform(X_moons_scaled)
    
    ax.scatter(X_kpca[:, 0], X_kpca[:, 1], 
               c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
    ax.set_title(f'gamma = {gamma}')
    ax.set_xlabel('KPC1')
    ax.set_ylabel('KPC2')

plt.suptitle('Effect of Gamma Parameter in RBF Kernel PCA', fontsize=14)
plt.tight_layout()
plt.show()

## 6. 超参数调优

使用网格搜索结合下游分类任务来选择最优的核函数和参数。

In [ ]:
# 划分训练集和测试集
X_train, X_test, y_train, y_test = train_test_split(
    X_moons_scaled, y_moons, test_size=0.2, random_state=42
)

# 创建 Pipeline：Kernel PCA + Logistic Regression
pipeline = Pipeline([
    ('kpca', KernelPCA(n_components=2)),
    ('clf', LogisticRegression(max_iter=1000))
])

# 参数网格
param_grid = {
    'kpca__kernel': ['rbf', 'poly', 'sigmoid'],
    'kpca__gamma': np.linspace(0.01, 0.1, 5)
}

# 网格搜索
grid_search = GridSearchCV(pipeline, param_grid, cv=5, scoring='accuracy', n_jobs=-1)
grid_search.fit(X_train, y_train)

print("最佳参数:")
print(f"  Kernel: {grid_search.best_params_['kpca__kernel']}")
print(f"  Gamma: {grid_search.best_params_['kpca__gamma']:.4f}")
print(f"\n交叉验证准确率: {grid_search.best_score_:.4f}")

# 测试集评估
y_pred = grid_search.predict(X_test)
print(f"测试集准确率: {accuracy_score(y_test, y_pred):.4f}")

## 7. 逆变换与重构

Kernel PCA 的逆变换是近似的，通过学习一个从降维空间到原始空间的映射来实现。

In [ ]:
# 启用逆变换
kpca_fit = KernelPCA(n_components=2, kernel='rbf', gamma=15, 
                     fit_inverse_transform=True, alpha=0.01)
X_kpca = kpca_fit.fit_transform(X_moons_scaled)

# 执行逆变换
X_reconstructed = kpca_fit.inverse_transform(X_kpca)

# 计算重构误差
mse = mean_squared_error(X_moons_scaled, X_reconstructed)
print(f"重构 MSE: {mse:.6f}")

# 可视化重构结果
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 原始数据
axes[0].scatter(X_moons_scaled[:, 0], X_moons_scaled[:, 1], 
                c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
axes[0].set_title('Original Data')
axes[0].set_xlabel('Feature 1')
axes[0].set_ylabel('Feature 2')

# 降维后的数据
axes[1].scatter(X_kpca[:, 0], X_kpca[:, 1], 
                c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
axes[1].set_title('Kernel PCA Projection')
axes[1].set_xlabel('KPC1')
axes[1].set_ylabel('KPC2')

# 重构的数据
axes[2].scatter(X_reconstructed[:, 0], X_reconstructed[:, 1], 
                c=y_moons, cmap='coolwarm', alpha=0.7, edgecolors='white')
axes[2].set_title(f'Reconstructed (MSE={mse:.4f})')
axes[2].set_xlabel('Feature 1')
axes[2].set_ylabel('Feature 2')

plt.tight_layout()
plt.show()

## 8. 分类性能对比

对比使用不同降维方法后的分类性能。

In [ ]:
# 定义不同的降维方法
methods = [
    ('No Reduction', None),
    ('PCA', PCA(n_components=2)),
    ('KPCA (linear)', KernelPCA(n_components=2, kernel='linear')),
    ('KPCA (rbf)', KernelPCA(n_components=2, kernel='rbf', gamma=15)),
    ('KPCA (poly)', KernelPCA(n_components=2, kernel='poly', gamma=10, degree=3))
]

results = []

for name, reducer in methods:
    if reducer is None:
        X_train_red, X_test_red = X_train, X_test
    else:
        X_train_red = reducer.fit_transform(X_train)
        X_test_red = reducer.transform(X_test)
    
    clf = LogisticRegression(max_iter=1000)
    clf.fit(X_train_red, y_train)
    
    train_acc = clf.score(X_train_red, y_train)
    test_acc = clf.score(X_test_red, y_test)
    
    results.append({
        'Method': name,
        'Train Accuracy': train_acc,
        'Test Accuracy': test_acc
    })
    
    print(f"{name:20s} | Train: {train_acc:.4f} | Test: {test_acc:.4f}")

# 可视化对比
fig, ax = plt.subplots(figsize=(10, 5))
x = np.arange(len(results))
width = 0.35

train_accs = [r['Train Accuracy'] for r in results]
test_accs = [r['Test Accuracy'] for r in results]

bars1 = ax.bar(x - width/2, train_accs, width, label='Train', color='steelblue')
bars2 = ax.bar(x + width/2, test_accs, width, label='Test', color='coral')

ax.set_ylabel('Accuracy')
ax.set_title('Classification Accuracy with Different Dimensionality Reduction Methods')
ax.set_xticks(x)
ax.set_xticklabels([r['Method'] for r in results], rotation=15, ha='right')
ax.legend()
ax.set_ylim(0.5, 1.05)

plt.tight_layout()
plt.show()

## 9. 总结

### 关键要点

1. **适用场景**：
   - 数据具有非线性结构
   - 标准 PCA 无法有效分离类别
   - 需要在高维特征空间中进行降维

2. **核函数选择**：
   - **RBF 核**：最常用，适合大多数非线性数据
   - **多项式核**：适合已知多项式关系的数据
   - **Sigmoid 核**：类似神经网络激活函数

3. **超参数调优**：
   - 通过交叉验证结合下游任务性能选择参数
   - gamma 控制核函数的局部性，需要仔细调节

4. **逆变换**：
   - 是近似的，不能完全恢复原始数据
   - 主要用于可视化和理解映射

### 注意事项

- 计算复杂度 $O(n^3)$，不适合大规模数据
- 核矩阵需要 $O(n^2)$ 内存
- 对于大数据考虑使用 Nyström 近似或随机特征方法